In [739]:
import numpy as np
import json
import random
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [740]:
# No. of agents
N = 8
# No. of available spots
s = 4

In [741]:
import matplotlib.colors as mcolors
named_colors = mcolors.CSS4_COLORS
color_names = list(named_colors.keys())
random.seed(1234)
random.shuffle(color_names)
shape_names = ['dot','diamond','square','triangle','star','triangleDown']

def bin_to_decimal(bin_str):
    return int(bin_str, 2)

def get_color(bin_str):
    decimal = bin_to_decimal(bin_str)
    return color_names[decimal % len(color_names)]

def get_shape(bin_str):
    decimal = bin_to_decimal(bin_str)
    return shape_names[decimal % len(shape_names)]

In [742]:
with open(f'graph_data_N{N:d}s{s:d}.json', 'r') as f:
  struct = json.load(f)
#print(json.dumps(struct, indent = 4))

In [743]:
Npat = 1
edges = []
e_colors = {}
n_colors = {}
n_shapes = {}
for n1 in struct[Npat]:
    n_colors[n1] = get_color(''.join(i for i in struct[Npat][n1]["pattern"]))
    n_shapes[n1] = get_shape(''.join(i for i in struct[Npat][n1]["pattern"]))
    for n2 in struct[Npat][str(n1)]["neigh"]:  
        if len(struct[Npat][str(n1)]["strat"]) > 1:
            edges.append((n2, n1))

            copy_strat = False
            if len(struct[Npat][str(n1)]["strat"]) == 2:
                copy_strat = (struct[Npat][str(n1)]["strat"]['0'] == '0') and (struct[Npat][str(n1)]["strat"]['1'] == '1')
                if copy_strat:
                    e_colors[f'{n2}-{n1}'] = 'darkturquoise'
                else:
                    e_colors[f'{n2}-{n1}'] = 'black'
            else:
                e_colors[f'{n2}-{n1}'] = 'black'

In [744]:
# print(edges)
# print(e_colors)
# print(n_colors)

In [745]:
n_colors['3']

'khaki'

In [746]:
DG = nx.DiGraph()
DG.add_nodes_from([str(i) for i in range(N)])
DG.add_edges_from(edges)
for node in DG.nodes:
    DG.nodes[node]['color'] = n_colors[node]

In [747]:
list(DG.nodes(data=True))

[('0', {'color': 'khaki'}),
 ('1', {'color': 'khaki'}),
 ('2', {'color': 'khaki'}),
 ('3', {'color': 'khaki'}),
 ('4', {'color': 'coral'}),
 ('5', {'color': 'coral'}),
 ('6', {'color': 'coral'}),
 ('7', {'color': 'coral'})]

In [748]:
dnet = Network("400px","400px",notebook=True,directed=True)
dnet.from_nx(DG)
for edge in dnet.edges:
    edge['color'] = e_colors[f"{edge['from']}-{edge['to']}"]
for node in dnet.nodes:
    node['color'] = {'background': n_colors[node['id']], 'border': 'black'}
    node['shape'] = n_shapes[node['id']]

In [749]:
dnet.toggle_physics(True)
dnet.show('dnet.html')

dnet.html


In [750]:
#degree = [len(list(DG.neighbors(str(i)))) for i in range(N)]
#degree

### Test
Check if by finding the maximum independent sets of the undirected graph one gets the same independent sets as in the directed graph.

Answer: Not necessarily.

In [751]:
#Maximal independent set of the undirected graph
UG = DG.to_undirected()
#nx.algorithms.mis.maximal_independent_set(UG)
colors_ = {}
coloring = nx.coloring.greedy_color(UG, strategy="independent_set")

In [752]:
coloring

{'0': 0, '3': 0, '4': 0, '5': 0, '7': 1, '1': 1, '6': 1, '2': 2}

In [753]:
n_colors[str(coloring['1'])]

'khaki'

In [754]:
dnet1 = Network("400px","400px",notebook=True,directed=False)
dnet1.from_nx(UG)
for node in dnet1.nodes:
    node['color'] = {'background': color_names[coloring[node['id']]], 'border': 'black'}
    #node['shape'] = n_shapes[node['id']]

In [755]:
dnet1.toggle_physics(True)
dnet1.show('dnet_u.html')

dnet_u.html


### Compute the graph entropy (UNDER CONSTRUCTION)

In [756]:
# A maximal independent set
maximal_is = nx.maximal_independent_set(UG, seed=42)
print(f"Maximal independent set: {maximal_is}")

Maximal independent set: ['1', '0', '7', '6']


In [757]:
# A Maximal Independent Set of a subgraph
subgraph = ['2', '1', '6']
maximal_is_subgraph = nx.maximal_independent_set(UG.subgraph(subgraph), seed=2)
print(f"Maximal independent set of the subgraph: {maximal_is_subgraph}")

Maximal independent set of the subgraph: ['2']


In [758]:
# Try to find all maximal independent sets
iss = []
for s in range(2*UG.number_of_nodes()):
    maximal_is = list(map(int, nx.maximal_independent_set(UG, seed=s)))
    maximal_is.sort()
    print(f"Maximal independent sets with seed {s}: {maximal_is}")
    iss.append(maximal_is)

Maximal independent sets with seed 0: [0, 1, 6, 7]
Maximal independent sets with seed 1: [2, 5, 7]
Maximal independent sets with seed 2: [0, 1, 3, 6]
Maximal independent sets with seed 3: [0, 3, 4, 5]
Maximal independent sets with seed 4: [0, 1, 3, 6]
Maximal independent sets with seed 5: [2, 3, 4, 5]
Maximal independent sets with seed 6: [0, 1, 6, 7]
Maximal independent sets with seed 7: [2, 3, 4, 5]
Maximal independent sets with seed 8: [0, 1, 3, 6]
Maximal independent sets with seed 9: [0, 1, 6, 7]
Maximal independent sets with seed 10: [0, 3, 4, 5]
Maximal independent sets with seed 11: [0, 1, 6, 7]
Maximal independent sets with seed 12: [0, 1, 6, 7]
Maximal independent sets with seed 13: [2, 3, 4, 5]
Maximal independent sets with seed 14: [0, 1, 3, 6]
Maximal independent sets with seed 15: [0, 1, 3, 6]


In [759]:
iss = np.array(iss)
np.unique(iss, axis=0)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16,) + inhomogeneous part.

----------------------
Understanding the independent subgraph algorithm for an undirected graph.

In [ ]:
seed_node = np.random.choice(UG.nodes()) # Pick a random node to start with

In [ ]:
neighbors = set(UG.adj[seed_node]) # Get the neighbors of the node
neighbors

{'0', '5'}

In [ ]:
indep_nodes = list(seed_node) # Initialize the independent set with the seed node
available_nodes = set(UG.nodes()).difference(neighbors.union(indep_nodes)) # Set of nodes that are not the seed nor its neighbors
available_nodes

{'1', '2', '3', '4', '6', '8'}

In [ ]:
while available_nodes:
    node = np.random.choice(list(available_nodes)) # Randomly select a node from the available nodes
    indep_nodes.append(node) # Add the selected node to the independent set
    available_nodes.difference_update(list(UG.adj[node]) + [node]) # Update the available nodes by removing the neighbors of the selected node and the node itself
indep_nodes

['7', np.str_('3'), np.str_('2'), np.str_('8')]

-----------------------------
Try the same thing for a directed graph

In [ ]:
seed_node = np.random.choice(DG.nodes()) # Pick a random node to start with

In [ ]:
neighbors = set(DG.adj[seed_node]) # Get the neighbors of the node
neighbors

{'7', '8'}

In [ ]:
indep_nodes = list(seed_node) # Initialize the independent set with the seed node
available_nodes = set(DG.nodes()).difference(neighbors.union(indep_nodes)) # Set of nodes that are not the seed nor its neighbors
available_nodes

{'0', '1', '2', '3', '4', '6'}

In [ ]:
while available_nodes:
    node = np.random.choice(list(available_nodes)) # Randomly select a node from the available nodes
    indep_nodes.append(node) # Add the selected node to the independent set
    available_nodes.difference_update(list(DG.adj[node]) + [node]) # Update the available nodes by removing the neighbors of the selected node and the node itself
indep_nodes

['5', np.str_('0'), np.str_('3'), np.str_('2'), np.str_('6')]